In [1]:
import geopandas as gpd
import numpy as np

In [ ]:
# load risk grid
grid = gpd.read_parquet("risk_grid.parquet")
grid["grid_id"] = grid.index

In [6]:
grid.head()

,geometry,Risk_Score,Crime_Count,Log_Risk_Score,grid_id
0,"POLYGON ((503771.5 155854.3, 503771.5 156054.3...",0.0,0.0,0.0,0
1,"POLYGON ((503771.5 156054.3, 503771.5 156254.3...",0.0,0.0,0.0,1
2,"POLYGON ((503771.5 156254.3, 503771.5 156454.3...",0.0,0.0,0.0,2
3,"POLYGON ((503771.5 156454.3, 503771.5 156654.3...",0.0,0.0,0.0,3
4,"POLYGON ((503771.5 156654.3, 503771.5 156854.3...",0.0,0.0,0.0,4


In [7]:
grid.sort_values("Risk_Score", ascending=False).head(10)


,geometry,Risk_Score,Crime_Count,Log_Risk_Score,grid_id
29731,"POLYGON ((529971.5 180854.3, 529971.5 181054.3...",22907.0,3221.0,10.039241,29731
29505,"POLYGON ((529771.5 180854.3, 529771.5 181054.3...",21103.0,2926.0,9.957218,29505
28828,"POLYGON ((529171.5 181054.3, 529171.5 181254.3...",20128.0,3117.0,9.909917,28828
29507,"POLYGON ((529771.5 181254.3, 529771.5 181454.3...",18506.0,2715.0,9.825904,29507
29732,"POLYGON ((529971.5 181054.3, 529971.5 181254.3...",16175.0,2306.0,9.691284,29732
29730,"POLYGON ((529971.5 180654.3, 529971.5 180854.3...",14484.0,2024.0,9.580869,29730
29504,"POLYGON ((529771.5 180654.3, 529771.5 180854.3...",10830.0,1488.0,9.290168,29504
28376,"POLYGON ((528771.5 181054.3, 528771.5 181254.3...",10640.0,1599.0,9.272470,28376
30181,"POLYGON ((530371.5 180454.3, 530371.5 180654.3...",9260.0,1239.0,9.133567,30181
28616,"POLYGON ((528971.5 183854.3, 528971.5 184054.3...",8782.0,1355.0,9.080573,28616


In [ ]:
# convertingh the geodataframe to a format suitable for database insertion
grid = grid.copy()

grid["geom_wkt"] = grid.geometry.apply(
    lambda g: f"SRID=27700;{g.wkt}"
)

grid = grid.drop(columns=["geometry"])

In [9]:
grid[["geom_wkt"]].head()

,geom_wkt
0,"SRID=27700;POLYGON ((503771.5 155854.3, 503771..."
1,"SRID=27700;POLYGON ((503771.5 156054.3, 503771..."
2,"SRID=27700;POLYGON ((503771.5 156254.3, 503771..."
3,"SRID=27700;POLYGON ((503771.5 156454.3, 503771..."
4,"SRID=27700;POLYGON ((503771.5 156654.3, 503771..."


In [24]:
# connect to database
import pymysql

In [25]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="root",
    database="londonsafetymap",
    autocommit=False,
)
cursor = conn.cursor()

In [28]:
records = list(
    zip(
        grid["grid_id"].astype(int),
        grid["Risk_Score"].astype(float),
        grid["Log_Risk_Score"].astype(float),
        grid["Crime_Count"].astype(int),
        grid["geom_wkt"]
    )
)


In [32]:
sql = """
INSERT INTO risk_grid
(grid_id, risk_score, log_risk_score, crime_count, geom_wkt)
VALUES (%s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
risk_score = VALUES(risk_score),
log_risk_score = VALUES(log_risk_score),
crime_count = VALUES(crime_count),
geom_wkt = VALUES(geom_wkt)
"""

cursor.executemany(sql, records)
conn.commit()
